<월 단위(결과 보고 어쩌면 주단위)>

- 과재고 상품 TOP20
    - 제품명/재고량/지난4주 주평균 판매량/재고회전주수(정렬기준)
- 부족재고 상품 TOP20
    - 제품명/마지막재고일/재고있을 때 평균 판매량(정렬기준)

In [1]:
from db_import import *
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 30)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Data Import
end_date = datetime.now().strftime('%Y-%m-%d')
query_obj = Queries('2022-01-01', end_date)
db_obj = DBImport(db_type='cscart')

analytics = db_obj.data_import(query_obj.analytics_query)
brand = db_obj.data_import(query_obj.brand_query)
inventory = db_obj.data_import(query_obj.inventory_query)
# category = db_obj.data_import(query_obj.category_query)
# mainexposure = db_obj.data_import(query_obj.mainexposure_query)

In [3]:
# basic preprocessing
analytics['purchased_ymd'] = pd.to_datetime(analytics.purchased_at).dt.normalize() # 시간 제외한 날짜만
inventory['sys_time'] = pd.to_datetime(inventory['sys_time']).dt.normalize() # 시간 제외한 날짜만
brand['brand'] = brand['brand'].apply(lambda x: x.strip())

In [4]:
# product-barcode matching table
prod_bar_match = analytics[['product_id', 'barcode']].drop_duplicates()

# barcode-product_name matching table (추후 바코드 없는 상품들은 바코드 기준 조인이 안되기 때문에 상품명, 옵션명 테이블 따로 생성)
bar_prod_name = analytics[['product_id', 'product_name_kor']].drop_duplicates()
bar_prod_name = bar_prod_name.groupby('product_id')['product_name_kor'].sum().reset_index() # 한 상품명 내에서 여러 옵션이 담겨있기도 함. 모든 옵션을 보여주기 위해 합치기

# barcode-variant_name matching table (추후 바코드 없는 상품들은 바코드 기준 조인이 안되기 때문에 상품명, 옵션명 테이블 따로 생성)
bar_var_name = analytics[['barcode', 'variant_1_name_kor']].drop_duplicates()
bar_var_name = bar_var_name.groupby('barcode')['variant_1_name_kor'].sum().reset_index() # 한 바코드 내에서 여러 옵션이 담겨있기도 함. 모든 옵션을 보여주기 위해 합치기

In [12]:
# 추후 컬럼명 한국어로 변경 시 공통 활용되는 dictionary
col_name_dict = {
    'barcode': '바코드',
    'amount': '현재고',
    'sys_time': '재고일자',
    'product_qty': '재고잔존 시 주평균판매수량',
    'wio': '재고소진까지 잔여기간(주)',
    'product_id': '상품번호',
    'brand': '브랜드',
    'product_name_kor': '상품명',
    'variant_1_name_kor': '옵션명',
    'last_stock_ymd': '최근재고보유일자',
    'oos_days': '결품경과일수'
}

### 과다재고 상품
- 제품명/재고량/지난4주 주평균 판매량/재고회전주수(정렬기준)

In [13]:
# 현재고
now_date = inventory['sys_time'].max()
now_inventory = inventory[inventory['sys_time']==now_date]

last_month_sales = analytics[(analytics['purchased_ymd']<=now_date)&(analytics['purchased_ymd']>(now_date-timedelta(days=28)))]
last_month_sales = last_month_sales.groupby(['barcode', 'purchased_ymd'])['product_qty'].sum().reset_index()
last_month_sales = last_month_sales.rename(columns={'purchased_ymd': 'sys_time'})
last_month_inventory = inventory[(inventory['sys_time']<=now_date)&(inventory['sys_time']>(now_date-timedelta(days=28)))] # 동일한 기간의 재고와 merge. 재고가 있는 기간의 판매량만 보기 위함(재고가 0인 경우 제외, 재고가 있는데 판매량이 집계되지 않는 경우 0으로 채우기 위함)
last_month_sales = pd.merge(last_month_inventory, last_month_sales, on=['barcode', 'sys_time'], how='outer')
last_month_sales = last_month_sales[~((last_month_sales['barcode'].isin(['', '0', 0]))|(last_month_sales['barcode'].isnull()))]
last_month_sales = last_month_sales.fillna(0)

# 지난4주 주평균 판매량
avg_week_sales = last_month_sales[last_month_sales['amount']>0].groupby('barcode')['product_qty'].mean().reset_index() # 재고가 있을 때 일평균 판매량
avg_week_sales['product_qty'] = np.ceil(avg_week_sales['product_qty'] * 7 ) # 일평균 판매량에 7곱하여 주평균 판매량 구한 후 ceiling

over_stock = pd.merge(now_inventory, avg_week_sales, on='barcode', how='left')
over_stock['wio'] = over_stock['amount'] / (over_stock['product_qty'].apply(lambda x: 1 if x==0 else x)) # week of inventory outstanding (재고자산 회전주수) # 평균판매량이 0일 경우, 1로 보정해줌. 비교기간동안 한 번도 팔리지 않았다는 의미인데, 그냥 둘 경우 inf가 나와서 비교가 어려움. 1로 넣고 재고 많은 순서대로 뜰 수 있도록. 그 외 추가로 주평균판매량인 product_qty는 그대로 살려둠으로써, 한번도 팔리지 않았는지, 주평균판매량이 1인지 구분은 할 수 있도록 작업
over_stock = over_stock.sort_values(by='wio', ascending=False)

# 후처리
over_stock = pd.merge(over_stock, prod_bar_match, on='barcode', how='left')
over_stock = pd.merge(over_stock, brand, on='product_id', how='left')
over_stock = pd.merge(over_stock, bar_prod_name, on='product_id', how='left') # barcode 없는 상품들로 인해 상품명과 옵션명 별도로 조인
over_stock = pd.merge(over_stock, bar_var_name, on='barcode', how='left')
over_stock = over_stock[[
    'product_name_kor', 'variant_1_name_kor', 'brand', 'wio', 'product_qty', 'amount'
]] # 컬럼 순서 변경
over_stock = over_stock.rename(columns=col_name_dict)

### 과소재고 상품
- 제품명/마지막재고일/재고있을 때 평균 판매량(정렬기준)

In [14]:
# 추후 부족재고 정보도 확인하기 위해 재고일자와 판매일자 merge
last_6month_sales = analytics[(analytics['purchased_ymd']<=now_date)&(analytics['purchased_ymd']>(now_date-timedelta(days=180)))]
last_6month_sales = last_6month_sales.groupby(['barcode', 'purchased_ymd'])['product_qty'].sum().reset_index()
last_6month_sales = last_6month_sales.rename(columns={'purchased_ymd': 'sys_time'})
last_6month_inventory = inventory[(inventory['sys_time']<=now_date)&(inventory['sys_time']>(now_date-timedelta(days=180)))] # 동일한 기간의 재고와 merge (재고가 0인 경우 제외, 재고가 있는데 판매량이 집계되지 않는 경우 0으로 채우기 위함)
last_6month_sales = pd.merge(last_6month_inventory, last_6month_sales, on=['barcode', 'sys_time'], how='outer')
last_6month_sales = last_6month_sales[~((last_6month_sales['barcode'].isin(['', '0', 0]))|(last_6month_sales['barcode'].isnull()))]
last_6month_sales = last_6month_sales.fillna(0)

# 주평균 판매량
avg_week_sales = last_6month_sales[last_6month_sales['amount']>0].groupby('barcode')['product_qty'].mean().reset_index() # 재고가 있을 때 일평균 판매량
avg_week_sales['product_qty'] = np.ceil(avg_week_sales['product_qty'] * 7 ) # 일평균 판매량에 7곱하여 주평균 판매량 구한 후 ceiling

# 마지막으로 재고 있었던 날
last_stock = inventory[inventory['amount']>0].groupby('barcode')['sys_time'].max().reset_index()
last_stock = last_stock.rename(columns={'sys_time': 'last_stock_ymd'})
under_stock = pd.merge(last_stock, avg_week_sales, on='barcode', how='left')
under_stock['product_qty'] = under_stock['product_qty'].fillna(0)
under_stock['oos_days'] = under_stock['last_stock_ymd'].apply(lambda x: (now_date - x).days)
under_stock = under_stock[under_stock['last_stock_ymd']!=now_date] # 현재고가 없는 상품들, 즉 결품 상품들에 대해서 집계
under_stock = under_stock.sort_values(by='product_qty', ascending=False)

# 후처리
under_stock = pd.merge(under_stock, prod_bar_match, on='barcode', how='left')
under_stock = pd.merge(under_stock, brand, on='product_id', how='left')
under_stock = pd.merge(under_stock, bar_prod_name, on='product_id', how='left') # barcode 없는 상품들로 인해 상품명과 옵션명 별도로 조인
under_stock = pd.merge(under_stock, bar_var_name, on='barcode', how='left')
under_stock = under_stock[[
    'product_name_kor', 'variant_1_name_kor', 'brand', 'product_qty', 'oos_days', 'last_stock_ymd'
]] # 컬럼 순서 변경
under_stock = under_stock.rename(columns=col_name_dict)

In [15]:
over_stock = over_stock.head(30)
under_stock = under_stock.head(30)

over_stock.index = over_stock.index + 1
under_stock.index = under_stock.index + 1

In [16]:
# file save
today = datetime.today().strftime('%Y%m%d')
path = '{}_과다과소재고.xlsx'.format(today)
writer = pd.ExcelWriter(path, engine='xlsxwriter')

over_stock.to_excel(writer, sheet_name='과다재고')
under_stock.to_excel(writer, sheet_name='과소재고')

writer.save()
writer.close()